In [1]:
from dataclasses import dataclass, field

In [2]:
@dataclass(frozen=True)
class DatasetConfig:
    IMAGE_SIZE:     int   = 1280
    BATCH_SIZE:     int   = -1
    CLOSE_MOSAIC:   int   = 10
    MOSAIC:         float = 0.3
    FLIP_LR:        float = 0.5 # Turn off horizontal flip.
    HSV_H:          float = 0.3
    HSV_S:          float = 0.3
    HSV_V:          float = 0.3
    ROTATION:       float = 0.2
    TRANSLATION:    float = 0.2
    SCALE:          float = 0.2
    SHEAR:          float = 0.2
    PERSPECTIVE:    float = 0
    FLIP_UD:        float = 0
    MIXUP:          float = 0
    ERASING:        float = 0.2
    CROP_FRACTION:  float = 0.2


In [3]:
@dataclass(frozen=True)
class TrainingConfig:
    DATASET_YAML:   str = "bone-keypoints.yaml"
    MODEL:          str = "yolov8m-pose.pt"
    EPOCHS:         int = 100
    KPT_SHAPE:    tuple = (11,3)
    PROJECT:        str = "Bone_keypoints"
    NAME:           str = f"{MODEL.split('.')[0]}_{EPOCHS}_epochs_no_aug"
    CLASSES_DICT:  dict = field(default_factory = lambda:{0 : "bone"})
    BOX_WEIGHT:     float = 1.5
    CLS_WEIGHT:     float = 0.1
    POSE_WEIGHT:    float = 20.0
    DROP_OUT:       float = 0.2

In [4]:
import os

DATA_DIR = "./bone-pose-data"
 
TRAIN_DIR         = f"train"
TRAIN_FOLDER_IMG    = f"images"
TRAIN_FOLDER_LABELS = f"labels"
 
TRAIN_IMG_PATH   = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_IMG)
TRAIN_LABEL_PATH = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_LABELS)
 
VALID_DIR           = f"valid"
VALID_FOLDER_IMG    = f"images"
VALID_FOLDER_LABELS = f"labels"
 
VALID_IMG_PATH   = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_IMG)
VALID_LABEL_PATH = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_LABELS)
 
os.makedirs(TRAIN_IMG_PATH, exist_ok=True)
os.makedirs(TRAIN_LABEL_PATH, exist_ok=True)
os.makedirs(VALID_IMG_PATH, exist_ok=True)
os.makedirs(VALID_LABEL_PATH, exist_ok=True)

In [5]:
train_config = TrainingConfig()
data_config = DatasetConfig()

In [6]:
print(data_config)

DatasetConfig(IMAGE_SIZE=1280, BATCH_SIZE=-1, CLOSE_MOSAIC=10, MOSAIC=0.3, FLIP_LR=0.5, HSV_H=0.3, HSV_S=0.3, HSV_V=0.3, ROTATION=0.2, TRANSLATION=0.2, SCALE=0.2, SHEAR=0.2, PERSPECTIVE=0.2, FLIP_UD=0, MIXUP=0.2, ERASING=0.2, CROP_FRACTION=0.2)


In [7]:
import yaml
current_dir = os.getcwd()
 
data_dict = dict(
                path      = os.path.join(current_dir, DATA_DIR),
                train     = os.path.join(TRAIN_DIR, TRAIN_FOLDER_IMG),
                val       = os.path.join(VALID_DIR, VALID_FOLDER_IMG),
                names     = train_config.CLASSES_DICT,
                kpt_shape = list(train_config.KPT_SHAPE),
               )
 
with open(train_config.DATASET_YAML, "w") as config_file:
    yaml.dump(data_dict, config_file)

In [8]:
from ultralytics import YOLO

pose_model = model = YOLO(train_config.MODEL)
 
pose_model.train(
            data         = train_config.DATASET_YAML,
            epochs       = train_config.EPOCHS,
            imgsz        = data_config.IMAGE_SIZE,
            batch        = data_config.BATCH_SIZE,
            project      = train_config.PROJECT,
            name         = train_config.NAME,
            box          = train_config.BOX_WEIGHT,
            cls          = train_config.CLS_WEIGHT,
            pose         = train_config.POSE_WEIGHT,
            hsv_h        = data_config.HSV_H,
            hsv_s        = data_config.HSV_S,
            hsv_v        = data_config.HSV_V,
            degrees      = data_config.ROTATION,
            translate    = data_config.TRANSLATION,
            scale        = data_config.SCALE,
            shear        = data_config.SHEAR,
            perspective  = data_config.PERSPECTIVE,
            flipud       = data_config.FLIP_UD,
            fliplr       = data_config.FLIP_LR,
            mosaic       = data_config.MOSAIC,
            mixup        = data_config.MIXUP,
            erasing      = data_config.ERASING,
            crop_fraction = data_config.CROP_FRACTION,
            dropout     = train_config.DROP_OUT
           )

New https://pypi.org/project/ultralytics/8.2.58 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.50 🚀 Python-3.11.6 torch-2.3.1 CUDA:0 (NVIDIA RTX A4000, 16102MiB)
engine/trainer: task=pose, mode=train, model=yolov8m-pose.pt, data=bone-keypoints.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=Bone_keypoints, name=yolov8m-pose_100_epochs_no_aug4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

train: Scanning /data/mah/boneyolo/bone-pose-data/train/labels.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]

WARNING ⚠️ No 'flip_idx' array defined in data.yaml, setting augmentation 'fliplr=0.0'



val: Scanning /data/mah/boneyolo/bone-pose-data/valid/labels.cache... 8 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]


Plotting labels to Bone_keypoints/yolov8m-pose_100_epochs_no_aug4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 83 weight(decay=0.0), 93 weight(decay=0.0004921875), 92 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to Bone_keypoints/yolov8m-pose_100_epochs_no_aug4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          8          8      0.572      0.125       0.24      0.175          0          0          0          0



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


      4/100      5.43G     0.2841      7.097     0.2396     0.5602      1.345          0       1280: 100%|██████████| 22/22 [00:04<00:00,  5.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 14.30it/s]

                   all          8          8      0.572      0.125       0.24      0.175          0          0          0          0


KeyboardInterrupt: 

In [ ]:
model.predict('./bone-pose-data/valid/images/7_Oo_FM97985_L_f_d.jpg', save=True, conf=0.9)


image 1/1 /data/mah/boneyolo/bone-pose-data/valid/images/7_Oo_FM97985_L_f_d.jpg: 448x640 1 bone, 7.2ms
Speed: 7.7ms preprocess, 7.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to Bone_keypoints/yolov8m-pose_100_epochs43


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'bone'}
 obb: None
 orig_img: array([[[153, 154, 152],
         [154, 155, 153],
         [157, 155, 154],
         ...,
         [125, 138, 146],
         [126, 138, 148],
         [128, 140, 150]],
 
        [[155, 156, 154],
         [153, 154, 152],
         [156, 154, 153],
         ...,
         [128, 141, 149],
         [127, 140, 148],
         [127, 139, 149]],
 
        [[154, 155, 153],
         [154, 155, 153],
         [153, 154, 152],
         ...,
         [128, 142, 148],
         [127, 140, 148],
         [128, 141, 149]],
 
        ...,
 
        [[152, 147, 144],
         [153, 148, 145],
         [153, 148, 145],
         ...,
         [132, 142, 149],
         [134, 143, 147],
         [133, 142, 146]],
 
        [[153, 148, 145],
         [152, 147, 144],
         [152, 147, 1